*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

# Combining Datasets: Merge and Join

In [2]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Categories of Joins

The ``pd.merge()`` function implements a number of types of joins: the *one-to-one*, *many-to-one*, and *many-to-many* joins.
All three types of joins are accessed via an identical call to the ``pd.merge()`` interface; the type of join performed depends on the form of the input data.
Here we will show simple examples of the three types of merges, and discuss detailed options further below.

### One-to-one joins

In [3]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [6]:
pd.concat([df1, df2], axis='columns')

,employee,group,employee,hire_date
0,Bob,Accounting,Lisa,2004
1,Jake,Engineering,Bob,2008
2,Lisa,Engineering,Jake,2012
3,Sue,HR,Sue,2014


In [4]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### Many-to-one joins

Many-to-one joins are joins in which one of the two key columns contains duplicate entries.
For the many-to-one case, the resulting ``DataFrame`` will preserve those duplicate entries as appropriate.
Consider the following example of a many-to-one join:

In [7]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

### Many-to-many joins

In [8]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


## Specification of the Merge Key

### The ``on`` keyword

Most simply, you can explicitly specify the name of the key column using the ``on`` keyword, which takes a column name or a list of column names:

In [18]:
df1['date'] = [2010, 2012, 2012, 2013]
df1

,employee,group,date
0,Bob,Accounting,2010
1,Jake,Engineering,2012
2,Lisa,Engineering,2012
3,Sue,HR,2013


In [19]:
df2.columns = ['employee', 'date']
df2

,employee,date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [23]:
pd.merge(df1, df2, on='employee', suffixes=['_creation', '_hiring'])

,employee,group,date_creation,date_hiring
0,Bob,Accounting,2010,2008
1,Jake,Engineering,2012,2012
2,Lisa,Engineering,2012,2004
3,Sue,HR,2013,2014


In [11]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

### The ``left_on`` and ``right_on`` keywords

At times you may wish to merge two datasets with different column names; for example, we may have a dataset in which the employee name is labeled as "name" rather than "employee".
In this case, we can use the ``left_on`` and ``right_on`` keywords to specify the two column names:

In [24]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group  date
0      Bob   Accounting  2010
1     Jake  Engineering  2012
2     Lisa  Engineering  2012
3      Sue           HR  2013

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  date  name  salary
0      Bob   Accounting  2010   Bob   70000
1     Jake  Engineering  2012  Jake   80000
2     Lisa  Engineering  2012  Lisa  120000
3      Sue           HR  2013   Sue   90000

In [25]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,date,salary
0,Bob,Accounting,2010,70000
1,Jake,Engineering,2012,80000
2,Lisa,Engineering,2012,120000
3,Sue,HR,2013,90000


### The ``left_index`` and ``right_index`` keywords

Sometimes, rather than merging on a column, you would instead like to merge on an index.
For example, your data might look like this:

In [33]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

df1a
                group  date
employee                   
Bob        Accounting  2010
Jake      Engineering  2012
Lisa      Engineering  2012
Sue                HR  2013

df2a
          date
employee      
Lisa      2004
Bob       2008
Jake      2012
Sue       2014

In [34]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

df1a
                group  date
employee                   
Bob        Accounting  2010
Jake      Engineering  2012
Lisa      Engineering  2012
Sue                HR  2013

df2a
          date
employee      
Lisa      2004
Bob       2008
Jake      2012
Sue       2014

pd.merge(df1a, df2a, left_index=True, right_index=True)
                group  date_x  date_y
employee                             
Bob        Accounting    2010    2008
Jake      Engineering    2012    2012
Lisa      Engineering    2012    2004
Sue                HR    2013    2014

In [38]:
df1a.join(df2a, lsuffix='_creation', rsuffix='_hiring')

,group,date_creation,date_hiring
employee,,,
Bob,Accounting,2010,2008
Jake,Engineering,2012,2012
Lisa,Engineering,2012,2004
Sue,HR,2013,2014


For convenience, ``DataFrame``s implement the ``join()`` method, which performs a merge that defaults to joining on indices:

In [30]:
display('df1a', 'df2a', 'df1a.join(df2a)')

ValueError: columns overlap but no suffix specified: Index(['date'], dtype='object')

ValueError: columns overlap but no suffix specified: Index(['date'], dtype='object')

If you'd like to mix indices and columns, you can combine ``left_index`` with ``right_on`` or ``left_on`` with ``right_index`` to get the desired behavior:

In [39]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

,group,date
employee,,
Bob,Accounting,2010
Jake,Engineering,2012
Lisa,Engineering,2012
Sue,HR,2013
,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


## Specifying Set Arithmetic for Joins

In [40]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

In [44]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Other options for the ``how`` keyword are ``'outer'``, ``'left'``, and ``'right'``.
An *outer join* returns a join over the union of the input columns, and fills in all missing values with NAs:

In [15]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0   Peter   fish   NaN
1    Paul  beans   NaN
2    Mary  bread  wine
3  Joseph    NaN  beer

The *left join* and *right join* return joins over the left entries and right entries, respectively.
For example:

In [16]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

## Overlapping Column Names: The ``suffixes`` Keyword

In [17]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name")
   name  rank_x  rank_y
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

In [18]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

df8
   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4

df9
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2

pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2

# Pandas and SQL

If data resides in a relational database, it is possible to query the data using SQL

In [45]:
# first install and import the database driver. 
# This depends on the database. For the sake of illustration we use
# the built-in sqlite database.
import sqlite3

In [46]:
# second, establish a connection
connection = sqlite3.connect('data/ClassicModels.db')

In [47]:
# third, query using SQL
# for example, 
pd.read_sql("SELECT * FROM Employees WHERE officeCode = 1", connection)

,employeeNumber,lastName,firstName,extension,email,reportsTo,jobTitle,officeCode
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,0,President,1
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1002,VP Sales,1
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1002,VP Marketing,1
3,1143,Bow,Anthony,x5428,abow@classicmodelcars.com,1056,Sales Manager (NA),1
4,1165,Jennings,Leslie,x3291,ljennings@classicmodelcars.com,1143,Sales Rep,1
5,1166,Thompson,Leslie,x4065,lthompson@classicmodelcars.com,1143,Sales Rep,1
